In [1]:
import torch
import numpy as np

import torchvision
from torchvision import datasets,transforms
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if(torch.cuda.is_available()):
  print("The device is " + torch.cuda.get_device_name())
else:
    print("Using cpu no gpu available")
print('\n',device)

The device is GeForce 940MX

 cuda:0


In [3]:
train = datasets.MNIST("",train = True ,download =  False, transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307, ), (0.3081, ))]))
test = datasets.MNIST("",train = False ,download = False, transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307, ), (0.3081, ))]))

C:\Users\leleg\anaconda3\envs\pytorchml\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
dtype = torch.float32

In [5]:
from torch.utils.data import random_split,DataLoader
train,val = random_split(train, [55000,5000])
train_loader = DataLoader(train,batch_size = 64)
val_loader = DataLoader(val,batch_size = 64)


In [6]:
test_loader = DataLoader(test,batch_size = 64)

In [7]:
for (x,y) in train_loader:
    print(x.shape,y.shape)
    break

torch.Size([64, 1, 28, 28]) torch.Size([64])


In [8]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

In [9]:
print_every = 100

In [10]:
metric = nn.CrossEntropyLoss()

In [11]:
class skipper(nn.Module):
    def __init__(self,width):
        super(skipper,self).__init__()
        #making a res block using sequential inside module hehe
        self.resb1 = nn.Sequential(
        nn.Conv2d(in_channels = width[0] , 
                  out_channels = width[1], 
                  kernel_size= (3,3),
                  stride = (1,1),
                  padding = 'same',
                  bias = True
                 ),
        nn.BatchNorm2d(width[1]),
        nn.ReLU(),
        nn.Conv2d(in_channels = width[1],
                  out_channels = width[2],
                  kernel_size = (3,3),
                  stride = (1,1),
                  padding = 'same',
                  bias = True
                 ),
        nn.BatchNorm2d(width[2]),
        nn.MaxPool2d(kernel_size = (2,2),stride = (2,2),padding = 0)
                  
        )
        self.skip = nn.Sequential(
        nn.Conv2d(in_channels = width[0],
                  out_channels = width[2],
                  kernel_size = (1,1),
                  padding = 'same',
                  bias = True
                 ),
        nn.BatchNorm2d(width[2]),
        nn.MaxPool2d(kernel_size = (2,2),stride = (2,2),padding = 0)
        )
    def forward(self,x):
        #cut = x
        out1 = self.resb1(x)
        #print(out1.shape)
        out2 = self.skip(x)
        x = F.relu(out1+out2)
        return x
        
        

In [12]:
class network(nn.Module):
    def __init__(self):
        super(network,self).__init__()
        self.block1 =  skipper(width= [1,16,32])
        #self.reduce = nn.MaxPool2d(kernel_size = (2,2),stride = (2,2),padding = 0)
        self.block2 =  skipper(width= [32,64,128])
        self.fc1    =  nn.Linear(128*7*7,200)
        self.fc2    =  nn.Linear(200,10)
        #self.bn1    =  nn.BatchNorm1d(200)
        
    def forward(self,x):
        out = self.block1(x)
        #print(out.shape)
        #out = self.reduce(out)
        out = self.block2(out)
        #out = self.reduce(out)
        out = flatten(out)
        out = self.fc1(out)
        out = F.relu(out)
        #out = self.bn1(out)
        score = self.fc2(out)
        return score

In [13]:
def check_accuracy_part(loader, model):
    print("VAL/TEST accuracy : ")   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [17]:
def train_part(model, optimizer,scheduler_f, epochs=1):
    losses =[]
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        avg = 0
        i = 0
        for t, (x, y) in enumerate(train_loader):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            #scores = scores.reshape(y.shape[1])
            #print(scores.shape,x.shape,y.shape)
            #break
            loss = metric(scores, y)
           

            optimizer.zero_grad()

        
            loss.backward()
             
        
            optimizer.step()
            #scheduler_f.step()
            avg+=float(loss)
            i+=1

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part(val_loader, model)
                print()
        scheduler_f.step()
        print('Epoch-{0} lr: {1}'.format(e, optimizer.param_groups[0]['lr']))
        losses.append(avg/i)
    #return losses
        #break
                     

In [18]:
learning_rate = 1e-3
model = network()
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size = 3,gamma = 0.3)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
              ReLU-3           [-1, 16, 28, 28]               0
            Conv2d-4           [-1, 32, 28, 28]           4,640
       BatchNorm2d-5           [-1, 32, 28, 28]              64
         MaxPool2d-6           [-1, 32, 14, 14]               0
            Conv2d-7           [-1, 32, 28, 28]              64
       BatchNorm2d-8           [-1, 32, 28, 28]              64
         MaxPool2d-9           [-1, 32, 14, 14]               0
          skipper-10           [-1, 32, 14, 14]               0
           Conv2d-11           [-1, 64, 14, 14]          18,496
      BatchNorm2d-12           [-1, 64, 14, 14]             128
             ReLU-13           [-1, 64, 14, 14]               0
           Conv2d-14          [-1, 128,

In [19]:


losses = train_part(model, optimizer,scheduler_f = scheduler,epochs = 8)

Iteration 0, loss = 2.3243
VAL/TEST accuracy : 
Got 536 / 5000 correct (10.72)

Iteration 100, loss = 0.1720
VAL/TEST accuracy : 
Got 4748 / 5000 correct (94.96)

Iteration 200, loss = 0.0773
VAL/TEST accuracy : 
Got 4825 / 5000 correct (96.50)

Iteration 300, loss = 0.1228
VAL/TEST accuracy : 
Got 4815 / 5000 correct (96.30)

Iteration 400, loss = 0.0459
VAL/TEST accuracy : 
Got 4868 / 5000 correct (97.36)

Iteration 500, loss = 0.0937
VAL/TEST accuracy : 
Got 4905 / 5000 correct (98.10)

Iteration 600, loss = 0.0745
VAL/TEST accuracy : 
Got 4900 / 5000 correct (98.00)

Iteration 700, loss = 0.0445
VAL/TEST accuracy : 
Got 4825 / 5000 correct (96.50)

Iteration 800, loss = 0.0850
VAL/TEST accuracy : 
Got 4876 / 5000 correct (97.52)

Epoch-0 lr: 0.001
Iteration 0, loss = 0.1698
VAL/TEST accuracy : 
Got 4893 / 5000 correct (97.86)

Iteration 100, loss = 0.0378
VAL/TEST accuracy : 
Got 4922 / 5000 correct (98.44)

Iteration 200, loss = 0.0495
VAL/TEST accuracy : 
Got 4902 / 5000 correct 

In [ ]:
torch.cuda.current_device() 

In [22]:
check_accuracy_part(train_loader,model)

VAL/TEST accuracy : 
Got 54978 / 55000 correct (99.96)


In [ ]:
torch.cuda.device_count() 

In [ ]:
torch.cuda.get_device_name()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
a = torch.rand((3,2,2))
b = torch.rot90(a,-1,[1,2])

In [ ]:
print(a,'\n\n\n',b)

In [ ]:
c,d  = torch.utils.data.random_split(range(10), [3, 7])

In [ ]:
for i in range(len(c)):
    print(c[i])